In [124]:
import os
import numpy as np
from scipy.signal import butter, iirnotch, lfilter
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd
from ecgdetectors import Detectors
import pyhrv.frequency_domain as fd
import pyhrv.time_domain as td
import pyhrv

In [125]:
# filtering
def filter_ecgsignal(ecg, fs):
    ecg_td = td.ecg(ecg, interactive= False, show = False, sampling_rate=fs)
    filtered_ecg = ecg_td['filtered']

    # t=np.arange(0,ecg.size*(1/fs),(1/fs))
    # t=t[:ecg.size]

    # plt.figure(figsize=(12,4))
    # plt.plot(t,ecg,label="raw ECG")
    # plt.plot(t,filtered_ecg, label="filtered ECG")
    # plt.xlabel('$Time (s)$') 
    # plt.ylabel('$ECG$') 
    # plt.legend(loc='upper left')

    return filtered_ecg

In [126]:
# detect R-peaks
def detect_rpeaks(fs, ecg):
    ecg_td = td.ecg(ecg, interactive= False, show = False, sampling_rate=fs)
    rpeaks = ecg_td['rpeaks']

    # plt.figure(figsize=(12,4))
    # plt.plot(filtered_ecg,label="ECG")
    # plt.plot(rpeaks,filtered_ecg[rpeaks], 'ro', label="R-peaks")
    # plt.xlabel('$Samples$') 
    # plt.ylabel('$ECG$') 
    # plt.legend(loc='upper left')
    return rpeaks

# **Feature extraction**

In [1]:
from hrv import HRV   # to import the module

# Feature extraction
def get_features(r_peaks_pan, fs, df, index):
    features = []

    # Time Domain Features
    hrv_class = HRV(fs)
    
    #Heart-rate feature extraction in BPM(HR)

    HR = hrv_class.HR(r_peaks_pan)
    features.append(np.mean(HR))
    features.append(np.min(HR))
    features.append(np.max(HR))
    features.append(np.std(HR))

    #Standard deviation of NN intervals(SDNN)
    features.append(hrv_class.SDNN(r_peaks_pan, normalise=True))

    #Standard deviation of the average of NN intervals(SDNN)
    features.append(hrv_class.SDANN(r_peaks_pan, normalise=True))

    #Root mean square of succesive differences(RMSSD)
    features.append(hrv_class.RMSSD(r_peaks_pan, normalise= True))

    # Standard deviation of succesive differences(SDSD)
    features.append(hrv_class.SDSD(r_peaks_pan))

    #The proportion of NN50 divided by total number of NNs(pNN50)
    features.append(hrv_class.pNN50(r_peaks_pan))

    #The proportion of NN20 divided by total number of NNs(pNN20)
    features.append(hrv_class.pNN20(r_peaks_pan))

    # Frequency domain features
    # get the frequency domain features of the ECG
    ecg_fft = fd.frequency_domain(rpeaks = r_peaks_pan, show = False)

    plt.close(ecg_fft["ar_plot"])
    plt.close(ecg_fft["lomb_plot"])
    plt.close(ecg_fft["fft_plot"])
    
    features.extend(ecg_fft['fft_abs'])
    features.append(ecg_fft['fft_total'])

    features.append(ecg_fft['fft_ratio'])
    features.extend(ecg_fft['fft_peak'])
    features.extend(ecg_fft['fft_norm'])

    # features.extend([ecg_fft['fft_peak'][0], ecg_fft['fft_peak'][1], ecg_fft['fft_peak'][2], ecg_fft['fft_norm'][0],
                    #  ecg_fft['fft_norm'][1], ecg_fft['fft_ratio'], ecg_fft['fft_total']])
    
    df.loc[index] = features
    return df

In [128]:
# get features from a signal and adds it to the dataframe
def get_ecgfeatures(ecg, fs, df, index):
    ecg = td.ecg(ecg, interactive= False, show = False, sampling_rate=700)
    rpeaks = ecg['rpeaks']
    filtered_ecg = ecg['filtered']
    # plt.figure(figsize=(12,4))
    # plt.plot(filtered_ecg)
    # plt.plot(rpeaks,filtered_ecg[rpeaks], 'ro')
    df_features = get_features(rpeaks, fs, df, index)
    return df_features